In [1]:
%load_ext autoreload
%autoreload 2

from spot.utils import proj_root, os
os.chdir(proj_root())

In [16]:
import requests
from spot.utils import proj_root

# Replace `example.py` with your desired Python source file.
with open(proj_root() / 'src/spot/model.py') as f:
    r = requests.post("https://type4py.com/api/predict?tc=0", f.read())

In [9]:
from spot.experiments.type4py import Type4PyResponseParser
from spot.static_analysis import FunctionSignature
pred_signatures = Type4PyResponseParser("spot.model").parse(r.json())
for path, sig in pred_signatures.items():
    if isinstance(sig, FunctionSignature):
        print(path)
        print("\t" + str(sig))

spot.model/dynamic_dataloader
	FuncSig((collate_fn: cst'torch.Tensor', dataset: cst'torch.Tensor', max_tokens: cst'int', shuffle: cst'Optional[bool]') -> cst'str')
spot.model/DecodingArgs.scale_ctx_size
	MethodSig((factor: cst'float') -> cst'List[str]')
spot.model/DecodingArgs.__repr__
	MethodSig(() -> cst'str')
spot.model/DatasetPredResult.accuracies
	MethodSig((common_type_names: cst'str') -> cst'str')
spot.model/DatasetPredResult.group_by_repo
	MethodSig(() -> cst'dict')
spot.model/ModelWrapper.scale_ctx_size
	MethodSig(() -> cst'str')
spot.model/ModelWrapper.decode_row
	MethodSig((n_labels: cst'int', row: cst'str') -> cst'str')
spot.model/ModelWrapper.predict_on_batch
	MethodSig((batch: cst'bool', num_return_sequences: cst'Callable[..., T]') -> cst'str')
spot.model/ModelWrapper.predict
	MethodSig((dataset: cst'numpy.ndarray', num_return_sequences: cst'int', tqdm_args: cst'bytes') -> cst'str')
spot.model/ModelWrapper.save_pretrained
	MethodSig((path: cst'List[str]') -> MISSING)
spot

In [15]:
from spot.static_analysis import PythonProject
from spot.function_dataset import accuracy_from_signatures
from spot.visualization import pretty_print_dict, assert_eq

ex_proj = PythonProject.from_root(proj_root(), file_filter=lambda f: f.name == "model.py")
label_signatures = {e.path: e.get_signature() for e in ex_proj.all_elems()}

pred_seq = []
label_seq = []
for path in label_signatures:
    label_seq.append(l_sig := label_signatures[path])
    pred_seq.append(p_sig := pred_signatures[path])

accs = accuracy_from_signatures(pred_seq, label_seq, set())
pretty_print_dict(accs)

partial_acc: 27.91% (count=43)
full_acc: 25.58% (count=43)
full_acc_by_common:
   rare: 25.58% (count=43)
full_acc_by_cat:
   FuncArg: 40.00% (count=15)
   FuncReturn: 11.11% (count=9)
   ClassAtribute: 21.05% (count=19)
full_acc_by_pos:
   range(0, 1): 20.69% (count=29)
   range(1, 2): 22.22% (count=9)
   range(2, 4): 60.00% (count=5)
avg_label_size: 1.0465
avg_pred_size: 1.0465
